## RaspberryPi 

Erstellt am 31.10.2019 für Raspberry Pi 3 Model B+.

# Ziel
**Temperaturmessung und Darstellung der Werte über ein Mobile durch ansurfen einer Web-Page auf dem RaspberryPi.**
![Temperaturmessung mit DS18B20](temp.JPG)


## 1. Installation
Mit SDformatter (geht auch für Windows10) die SD-Karte formatieren.

**Installation dauert über 1 Stunde!** Von [www.raspberrypi.org](http://www.raspberrypi.org) aktuelles NOOBS „Network install only“ (nicht offline) downloaden. Voraussetzung ist Netzwerk-Zugriff. Anschliessend auf SD-Karte entpacken.

Dann SD-Karte in RaspberryPi einsetzen und starten. Zuerst Sprache Deutsch und Tastatur CH eingeben, dann WLAN verbinden. **Raspbian mit Desktop** ankreuzen. Achte auf den Speicherbedarf, es ist nicht unbedingt die volle Installation notwendig.Dann installieren und den Installationsanweisungen folgen.
 
Folgende Einstellungen bei Raspberry-Pi-Konfiguration vornehmen, über **Einstellungen>Raspberry-Pi-Konfiguration>Schnittstellen** mit folgenden Anpassungen:
 - Auflösung festlegen, sodass diese auch über Remote festgeleft ist *2)* 
 - aktivieren SSH (für Datei-Zugriff mit SFTP)
 - aktivieren VNC (für Remote-Desktop-Zugriff)
 - aktivieren I2C (für GPIO-Schnittstelle zum BME680 Sensor)
 - aktivieren Serial Port (für Modbus-Kommunikation mit RS485)
 - DEaktivieren Serial Console


Anschliessend werden einige Programme installiert (dauert > 30 Minuten). Hierfür verwenden wir den  Linux-Package-Installer "apt-get" für die Entwicklungsumgebung Spyder und pil (Python Image Library) für das Display:

    sudo apt-get install spyder3 -y
    sudo apt-get install python3-pil

Für die Python-Bibliotheken verwenden wir den Python-Package-Installer "pip3" (Python3):

    sudo pip3 install django==2.2.5
    sudo pip3 install bokeh==1.3.4
    sudo pip3 install pandas==0.25.2
    sudo pip3 install pyserial==3.4
    sudo pip3 install adafruit-circuitpython-ssd1306
    
Wir arbeiten mit Python 3.x. Um Bibliotheken hierfür zu installieren muss darauf geachtet werden das die "3"-Version verwendet wird. Z.B. **pip3 anstatt pip** oder spyder3 anstatt spyder. Ohne "3" wird die Version für Python 2.x verwendet. **Der Raspberry benötigt für das System beide Versionen!** Standardmässig wird Python2.x verwendet. Erkennbar durch "python --version", oder man startet auf der Console "python" und führt die Division 3/2 durch. Python2.x liefert 1. Wird auf der Konsole "python3" gestartet ergibt 3/2 = 1.5.

*2)* Hinweis Raspberry Pi4: Auflösung bei nicht angeschlossenem Monitor (headless) über VNS kann nichtmehr im Menü eingestellt werden. Dies muss in der /boot/config.txt Datei erfolgen mittels:

    hdmi_force_hotplug=1 
    hdmi_group=2
    hdmi_mode=0x45 
see: https://www.raspberrypi.org/forums/viewtopic.php?f=28&t=5851


### Beispielprojekt installieren
Das funktionierende Beispiel ist auf [github.com/markstaler/daqProjekt](https://github.com/markstaler/daqProjekt) abgelegt. Wir beziehen dies direkt von Github in "home/pi" Verzeichnis. Hierzu gehen wir in diesen Ordner (ist der Standard beim Öffnen der Konsole) und geben folgendes ein:

    git clone https://github.com/markstaler/daqProjekt
    
    

### Autostart Display
Im Projektverzeichnis "daqProject" ist bereits die Start-Datei "daqStart.py" für das Display enthalten. Nun konfigurieren wir es so, dass es beim booten des Raspberry gestartet wird. Dies wird in der Datei /etc/rc.local definiert. Wir öffnen den Editor (geany) mit Administrationsrechte:

    sudo geany
    
Anschliessend gehen wir in den Ordner "/etc" und öffnen die Datei "rc.local". Zuunderst in der Datei ergänzen wir **vor** "exit 0" unseren Startbefehl "sudo python3 /home/pi/daqProjekt/daqStart.py &".
Das "&" Zeichen am Ende bedeutet dass das Programm im Hintergrund läuft.


## WLAN Einstellung
Die WLAN-Verbindung definieren wir direkt in der Konfigurationsdatei **/etc/wpa_supplicant/wpa_supplicant.conf**.
Wir können hierfür den Editor nano auf der Konsole verwenden durch folgenden aufruf:
    
    sudo nano /etc/wpa_supplicant/wpa_supplicant.conf

oder wir öffnen den Editor "sudo geany" auf der Oberfläche. In der Datei führen wir alle WLAN-Netzwerke an, wie z.B. das eduroam oder das WLAN-Netz zuhause.

    ctrl_interface=DIR=/var/run/wpa_supplicant GROUP=netdev
    update_config=1
    country=CH

    network={
                ssid="wlan_zuHause"
                psk="passwort"
    }
    
    network={
                ssid="gast-ntb"
                psk="passwort"
                key_mgmt=WPA-PSK
                priority=1
    }    
    
    network={
                ssid="eduroam"
                scan_ssid=1
                key_mgmt=WPA-EAP
                eap=PEAP
                identity="name@ntb.ch"
                password="abcd"
    }
